<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Juypter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Numerical methods: 7. Linear systems
## Iterative Jacobi method
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

In this notebook, we implement the iterative **Jacobi method** as a method to solve 
a system of linear equations.

In [ ]:
import numpy as np

----
## Jacobi method

We need to define two steps for the method.

1. **Initial step**

Define an initial guess for the solution, ${\bf x^{(0)}}$.

2. **Iterative steps**

Find improved **solution vector** ${\bf x^{(k)}}$ from successive iterations:
$$
x_i^{(k)} = \frac{1}{a_{ii}} \left( b_i - \sum\limits_{j=1,i \ne j}^{n} a_{ij} x_j^{(k-1)} \right),
 \quad i=1,n
$$

Examine in `python`:

In [ ]:
def lin_jacobi(a,b,x,tol=0.001):
    '''
    !----------------------------------------------------------------------
    ! subroutine solves the system of linear equations
    ! a(n,n)*x(n) = b(n)
    ! using the Jacobi algorithm
    ! Input:
    ! n       - number of equations
    ! a(n,n)  - coefficient matrix
    ! b(n)    - rhs vector
    ! x(n)    - guess for solution vector
    ! tol     - accuracy of solution
    ! Output:
    ! x(n)    - improved solution vector
    ! Fixed:
    ! itmax   - max. number of iterations
    ! (err)   - type of norm used 
    ! (c) Georg Kaufmann
    !----------------------------------------------------------------------
    '''
    n = len(b)
    err = tol+1
    xold = np.zeros_like(x)
    for i in range(n):
        xold[i] = x[i]
    #print(0,err,x)
    
    it = 0
    while (err >= tol):
        err = 0
        it += 1
        for i in range(n):
            res = b[i]
            for j in range(n):
                if (i is not j):
                    res = res - a[i][j] * xold[j]
            x[i] = res / a[i][i]
            err  = err + (x[i]-xold[i])**2
        for i in range(n):
            xold[i] = x[i]
        if (err != 0):
            err = np.sqrt(err)
        #print(it,err,x)
        if (it > 100):
            return x
    return x

In [ ]:
def lin_jacobi2(A,b,x,tol):
    """Solves the equation Ax=b via the Jacobi iterative method."""
    # set error flag to false and initialize counter
    err = False
    it = 0
    # exteact diagonal entries D from A
    # then calculate resudual matrix R=A-D
    D = np.diag(A)
    R = A - np.diagflat(D)
    # solve Jakobi iteration, check for tolerance
    while (not err):
        it += 1
        x = (b - np.dot(R,x)) / D
        err = np.allclose(np.dot(A, x), b,rtol=tol,atol=tol)
        print(it,err,x)
    return x

## Examples

We test the **Gauss elimination method** with two examples:

1. ${\bf A} {\bf x} = {\bf b}$

with 
$$
{\bf A} = \left[
\begin{array}{ccc}
2&1&-1 \\ 1&3&1 \\ -1&1&4
\end{array}
\right];
{\bf b} = \left[
\begin{array}{c} 4 \\ 3 \\ 4 \end{array}
\right]
$$
The solution vector is ${\bf x} \simeq (3.46,-0.85,2.08)$.

In [ ]:
a=np.array([[2.,1.,-1.],[1.,3.,1.],[-1.,1.,4.]],dtype=float)
b=np.array([4.,3.,4.],dtype=float)
x=np.array([0.,0.,0.],dtype=float)
tol = 0.0001

In [ ]:
a2=np.copy(a); b2=np.copy(b); x2=np.copy(x)
print('A_ij: \n',a,a.ndim,a.shape)
print('b_j:  \n',b,b.ndim,b.shape)
x=lin_jacobi(a2,b2,x2,tol=tol)
print('x_i:  \n',x,x.ndim,x.shape)
print(np.allclose(np.dot(a, x),b,rtol=tol,atol=tol))

a2=np.copy(a); b2=np.copy(b); x2=np.copy(x)
x=lin_jacobi2(a2,b2,x2,tol=tol)
print('x_i:  \n',x,x.ndim,x.shape)
print(np.allclose(np.dot(a, x),b,rtol=tol,atol=tol))

# check against numpy solution
a2=np.copy(a); b2=np.copy(b)
x = np.linalg.solve(a2, b2)
print('x_i:  \n',x,x.ndim,x.shape)
print(np.allclose(np.dot(a, x), b))

2. ${\bf A} {\bf x} = {\bf b}$

with 
$$
{\bf A} = \left[
\begin{array}{cccc}
10&-1&2&0\\
-1&11&-1&3\\
 2&-1&10&-1\\
 0&3&-1&8
\end{array} 
\right];
{\bf b} = \left[
\begin{array}{c} 6 \\ 25 \\ -11 \\ 15 \end{array}
\right]
$$
The solution vector is ${\bf x} = (−1,2,-1,1)$.

In [ ]:
a=np.array([[10,-1,2,0],[-1,11,-1,3],[2,-1,10,-1],[0,3,-1,8]],dtype=float)
b=np.array([6,25,-11,15],dtype=float)
x=np.array([0.,0.,0.,0.],dtype=float)
tol = 0.0001

In [ ]:
print(np.diag(a))
print(np.diagflat(np.diag(a)))
print(np.diag(np.diag(a)))

In [ ]:
a2=np.copy(a); b2=np.copy(b); x2=np.copy(x)
print('A_ij: \n',a,a.ndim,a.shape)
print('b_j:  \n',b,b.ndim,b.shape)
x=lin_jacobi(a2,b2,x2,tol=tol)
print('x_i:  \n',x,x.ndim,x.shape)
print(np.allclose(np.dot(a, x),b,rtol=tol,atol=tol))

a2=np.copy(a); b2=np.copy(b); x2=np.copy(x)
x=lin_jacobi2(a2,b2,x2,tol=tol)
print('x_i:  \n',x,x.ndim,x.shape)
print(np.allclose(np.dot(a, x),b,rtol=tol,atol=tol))

# check against numpy solution
a2=np.copy(a); b2=np.copy(b)
x = np.linalg.solve(a2, b2)
print('x_i:  \n',x,x.ndim,x.shape)
print(np.allclose(np.dot(a, x), b))

3. ${\bf A} {\bf x} = {\bf b}$

with 
$$
{\bf A} = \left[
\begin{array}{cccc}
  1 &  1 &  0 &  3 \\
  2 &  1 & -1 &  1 \\
  3 & -1 & -1 &  2 \\
 -1 &  2 &  3 & -1 
\end{array} 
\right];
{\bf b} = \left[
\begin{array}{c} 4 \\ 1 \\ -3 \\ 4 \end{array}
\right]
$$
The solution vector is ${\bf x} = (−1,2,0,1)$.

In [ ]:
a=np.array([[1.,1.,0.,3.],[2.,1.,-1.,1.],[3.,-1.,-1.,2.],[-1.,2.,3.,-1.]])
b=np.array([4.,1.,-3.,4.])
x=np.array([0.,0.,0.,0.])

# geht nicht ...

[next>](Numerics_lab07_lu.ipynb)